plt.title('Variable Importance')# Pipeline for Data Pre-processing

In [1]:
import pickle
import helpers
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer as SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn_pandas import CategoricalImputer
from scipy import sparse

In [2]:
df = pickle.load(open('../data/df.pkl', 'rb'))
num_attribs = pickle.load(open('../data/num_attribs.pkl', 'rb'))
cat_attribs = pickle.load(open('../data/cat_attribs.pkl', 'rb'))

## Pipeline for imputing and standardizing data

Since the steps are very standard in almost any regression problem, I won't write too much explanation here.

In [3]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', CategoricalImputer(strategy='constant', fill_value='nan')), # mode by default; only works with string values
        ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ])

df = df[num_attribs + cat_attribs]
cols_contain_nans = df.columns[df.isna().any()].tolist()
cat_cols_contain_nans = [col for col in cols_contain_nans if col in cat_attribs]
print("cat_cols_contain_nans: \n", cat_cols_contain_nans)

cat_cols_contain_nans: 
 ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'host_response_time', 'cancellation_policy']


c:\users\zack\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
# to make sure the dependent variable is not in the training data
try:
    num_attribs.remove("log_price")
except:
    pass

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs)
])

In [5]:
X = full_pipeline.fit_transform(df[num_attribs + cat_attribs])
y = df["log_price"]

c:\users\zack\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## Save the Data

In [6]:
sparse.save_npz("../data/X.npz", X)
var_names = helpers.get_transformer_feature_names(full_pipeline)
pickle.dump(var_names, open('../data/var_names.pkl', 'wb'))
y.to_csv("../data/y.csv")


c:\users\zack\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.
